In [1]:
from sys import path
path.append("../")

# Exporting Landsat 7 to GeoTiff format  

> **Description**  
> The code in this notebook subsets a data cube, selects a specific set of variables, and then outputs that data into a GeoTIFF file. The goal is to be able to do external analyses of this data using other data analysis tools or GIS tools. The files would be reasonable in size, since we would restrict the region and parameters in the output.

----  

# Boiler Plate, Loading Data

> ### Import the Datacube

In [2]:
import datacube
dc = datacube.Datacube(app = 'my_app', config = '/home/ksaadmin/.datacube.conf')

>### Browse the available Data Cubes on the storage platform    
> You might want to learn more about what data is stored and how it is stored.


In [3]:
list_of_products = dc.list_products()
list_of_products

,name,description,license,default_crs,default_resolution
name,,,,,
ls8_usgs_level1_scene,ls8_usgs_level1_scene,Landsat 8 USGS Level 1 Collection-1 OLI-TIRS,None,None,None
usgs_ls7e_level1_1,usgs_ls7e_level1_1,United States Geological Survey Landsat 7 Enha...,CC-BY-4.0,None,None
usgs_ls8e_level1_1,usgs_ls8e_level1_1,Landsat 8 USGS Level 1 Collection-1 OLI-TIRS,CC-BY-4.0,None,None


>### Pick a product  
>Use the platform names from the previous block to select a small Data Cube. The data_access_api utility will give you lat, lon, and time bounds of your Data Cube.   

In [4]:
product = "usgs_ls7e_level1_1"
platform = "LANDSAT_7"
data_full = dc.load(product=product,output_crs ='EPSG:4326', resolution=(-0.00027,0.00027))
#obtain latitude extends
latitude_extents = data_full.latitude.values
size = latitude_extents.size-1
latitude_extents = (latitude_extents[0],latitude_extents[size])

#obtain longitude extends
longitude_extents = data_full.longitude.values
size = longitude_extents.size-1
longitude_extents = (longitude_extents[0],longitude_extents[size])
#print 
print(latitude_extents)
print(longitude_extents)

(-3.3835050000000004, -5.2929450000000005)
(38.470815, 40.611915)


# Visualize Data Cube Region

In [5]:
## The code below renders a map that can be used to orient yourself with the region.
from utils.data_cube_utilities.dc_display_map import display_map
display_map(latitude = latitude_extents, longitude = longitude_extents)

In [6]:
# Select an analysis region (Lat-Lon) within the extents listed above. 
longitude_extents = (38.5, 39.6)
latitude_extents = (-3.5, -4.6) 
# Time Period
time_extents = ('2000', '2022')

In [7]:
from utils.data_cube_utilities.dc_display_map import display_map

display_map(latitude = latitude_extents, longitude = longitude_extents)

In [8]:
# Filter the image
query = {
    'time' : time_extents,
    'latitude' : latitude_extents,
    'longitude' : longitude_extents,
}

#load dataset
data_partial = dc.load(product = product,
                          measurements = ['red', 'green', 'blue', 'nir', 'swir1', 'swir2', 'quality'],
                          output_crs='EPSG:4326', resolution=(-0.00027,0.00027),
                          **query
                         )

/home/ksaadmin/miniconda3/envs/cubeenv/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class CommonTimestamp will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  return self._connection.execute(select_query)
/home/ksaadmin/miniconda3/envs/cubeenv/lib/python3.9/site-packages/datacube/drivers/postgres/_api.py:564: SAWarning: Class Float8Range will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance de

In [9]:
data_partial

<xarray.Dataset>
Dimensions:      (time: 4, latitude: 4075, longitude: 4075)
Coordinates:
  * time         (time) datetime64[ns] 2000-12-07T07:21:56.518157 ... 2006-02...
  * latitude     (latitude) float64 -3.5 -3.5 -3.5 -3.501 ... -4.599 -4.6 -4.6
  * longitude    (longitude) float64 38.5 38.5 38.5 38.5 ... 39.6 39.6 39.6 39.6
    spatial_ref  int32 4326
Data variables:
    red          (time, latitude, longitude) uint16 55537 55537 55537 ... 28 27
    green        (time, latitude, longitude) uint16 55537 55537 55537 ... 39 39
    blue         (time, latitude, longitude) uint16 55537 55537 55537 ... 74 74
    nir          (time, latitude, longitude) uint16 55537 55537 55537 ... 11 11
    swir1        (time, latitude, longitude) uint16 55537 55537 55537 ... 12 11
    swir2        (time, latitude, longitude) uint16 55537 55537 55537 ... 11 10
    quality      (time, latitude, longitude) uint16 0 0 0 0 ... 672 672 672 672
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

# Derive Several Products

>### Unpack quality layer

In [10]:
import xarray as xr  
import numpy as np

def ls7_unpack_qa( data_array , cover_type):  
    
    land_cover_endcoding = dict( fill     =  [1], 
                                 clear    =  [66,  130], 
                                 water    =  [68,  132],
                                 shadow   =  [72,  136],
                                 snow     =  [80,  112, 144, 176],
                                 cloud    =  [96,  112, 160, 176, 224],
                                 low_conf =  [66,  68,  72,  80,  96,  112],
                                 med_conf =  [130, 132, 136, 144, 160, 176],
                                 high_conf=  [224]
                               ) 
    boolean_mask = np.isin(data_array.values, land_cover_endcoding[cover_type]) 
    return xr.DataArray(boolean_mask.astype(int),
                        coords = data_array.coords,
                        dims = data_array.dims,
                        name = cover_type + "_mask",
                        attrs = data_array.attrs)  

In [11]:
clear_xarray  = ls7_unpack_qa(data_partial.quality, "clear")  
water_xarray  = ls7_unpack_qa(data_partial.quality, "water")

shadow_xarray = ls7_unpack_qa(data_partial.quality, "shadow")  

In [12]:
clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")

clean_mask = np.logical_or(clear_xarray.values.astype(bool),
                           water_xarray.values.astype(bool)) 

/tmp/ipykernel_4060832/2824950681.py:1: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  clean_xarray = xr.ufuncs.logical_or(clear_xarray , water_xarray).astype(np.int8).rename("clean_mask")
/home/ksaadmin/miniconda3/envs/cubeenv/lib/python3.9/site-packages/xarray/core/dataarray.py:3098: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  f(self.variable, other_variable)
/home/ksaadmin/miniconda3/envs/cubeenv/lib/python3.9/site-packages/xarray/core/variable.py:2467: FutureWarning: xarray.ufuncs is deprecated. Instead, use numpy ufuncs directly.
  f(self_data, other_data) if not reflexive else f(other_data, self_data)


> ### Water

In [13]:
from utils.data_cube_utilities.dc_water_classifier import wofs_classify

water_classification = wofs_classify(data_partial,
                                     clean_mask = clean_mask, 
                                     mosaic = False) 

/home/ksaadmin/Datacube/datacube-notebooks/Export Tutorial/../utils/data_cube_utilities/dc_water_classifier.py:117: RuntimeWarning: invalid value encountered in true_divide
  return (a - b) / (a + b)


In [14]:
wofs_xarray = water_classification.wofs

> ###  Normalized Indices  

In [15]:
def NDVI(dataset):
    return ((dataset.nir - dataset.red)/(dataset.nir + dataset.red)).rename("NDVI")

In [16]:
def NDWI(dataset):
    return ((dataset.green - dataset.nir)/(dataset.green + dataset.nir)).rename("NDWI")

In [17]:
def NDBI(dataset):
        return ((dataset.swir2 - dataset.nir)/(dataset.swir2 + dataset.nir)).rename("NDBI")

In [18]:
ndbi_xarray = NDBI(data_partial)  # Urbanization - Reds
ndvi_xarray = NDVI(data_partial)  # Dense Vegetation - Greens
ndwi_xarray = NDWI(data_partial)  # High Concentrations of Water - Blues  

>### TSM  

In [19]:
from utils.data_cube_utilities.dc_water_quality import tsm

tsm_xarray = tsm(data_partial, clean_mask = wofs_xarray.values.astype(bool) ).tsm

> ### EVI  

In [20]:
def EVI(dataset, c1 = None, c2 = None, L = None):
        return ((dataset.nir - dataset.red)/((dataset.nir  + (c1 * dataset.red) - (c2 *dataset.blue) + L))).rename("EVI")

In [21]:
evi_xarray = EVI(data_partial, c1 = 6, c2 = 7.5, L = 1 )

# Combine Everything  

In [22]:
combined_dataset = xr.merge([data_partial,
          clean_xarray,
          clear_xarray,
          water_xarray,
          shadow_xarray,
          evi_xarray,
          ndbi_xarray,
          ndvi_xarray,
          ndwi_xarray,
          wofs_xarray,
          tsm_xarray])

# Copy original crs to merged dataset 
combined_dataset = combined_dataset.assign_attrs(data_partial.attrs)

combined_dataset

<xarray.Dataset>
Dimensions:      (time: 4, latitude: 4075, longitude: 4075)
Coordinates:
  * time         (time) datetime64[ns] 2000-12-07T07:21:56.518157 ... 2006-02...
  * latitude     (latitude) float64 -3.5 -3.5 -3.5 -3.501 ... -4.599 -4.6 -4.6
  * longitude    (longitude) float64 38.5 38.5 38.5 38.5 ... 39.6 39.6 39.6 39.6
    spatial_ref  int32 4326
Data variables: (12/17)
    red          (time, latitude, longitude) float32 5.554e+04 ... 27.0
    green        (time, latitude, longitude) float32 5.554e+04 ... 39.0
    blue         (time, latitude, longitude) float32 5.554e+04 ... 74.0
    nir          (time, latitude, longitude) float32 5.554e+04 ... 11.0
    swir1        (time, latitude, longitude) float32 5.554e+04 ... 11.0
    swir2        (time, latitude, longitude) float32 5.554e+04 ... 10.0
    ...           ...
    EVI          (time, latitude, longitude) float32 -0.0 -0.0 ... 0.04199
    NDBI         (time, latitude, longitude) float32 0.0 0.0 ... 0.0 -0.04762
    NDVI         (time, latitude, longitude) float32 0.0 0.0 ... -0.4359 -0.4211
    NDWI         (time, latitude, longitude) float32 0.0 0.0 0.0 ... 0.56 0.56
    wofs         (time, latitude, longitude) float64 -9.999e+03 ... -9.999e+03
    tsm          (time, latitude, longitude) float64 6.454e+04 ... 0.3705
Attributes:
    crs:           EPSG:4326
    grid_mapping:  spatial_ref

# Export Geotiff

----  
File formatting  

In [23]:
import time
def time_to_string(t):
    return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))

----  
This function can be used to write a single time slice from an xarray to geotiff

In [24]:
from utils.data_cube_utilities import dc_utilities
def export_slice_to_geotiff(ds, path):
    dc_utilities.write_geotiff_from_xr(path,
                                        ds.astype(np.float32),
                                        list(combined_dataset.data_vars.keys()),
                                        crs="EPSG:4326")

----  
For each time slice in a dataset we call `export_slice_to_geotif`  

In [25]:
def export_xarray_to_geotiff(ds, path):
    for t in ds.time:
        time_slice_xarray = ds.sel(time = t)
        export_slice_to_geotiff(time_slice_xarray,
                                path + "_" + time_to_string(t) + ".tif")

----  
Start Export

In [26]:
export_xarray_to_geotiff(combined_dataset, "geotiffs/landsat7")

/tmp/ipykernel_4060832/1726003935.py:3: DeprecationWarning: an integer is required (got type DataArray).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))
/tmp/ipykernel_4060832/1726003935.py:3: DeprecationWarning: an integer is required (got type DataArray).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))
/tmp/ipykernel_4060832/1726003935.py:3: DeprecationWarning: an integer is required (got type DataArray).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return time.strftime("%Y_%m_%d_%H_%M_%S", time.gmtime(t.astype(int)/1000000000))
/tmp/ipykernel_4060832/1726003935.py:3: DeprecationWarning: an integer is required (got type DataArr

----  
Sanity check using `gdalinfo` to make sure that all of our bands exist    

In [27]:
!gdalinfo geotiffs/landsat7_2000_12_07_07_21_56.tif

Driver: GTiff/GeoTIFF
Files: geotiffs/landsat7_2000_12_07_07_21_56.tif
Size is 4075, 4075
Coordinate System is:
GEOGCRS["WGS 84",
    DATUM["World Geodetic System 1984",
        ELLIPSOID["WGS 84",6378137,298.257223563,
            LENGTHUNIT["metre",1]]],
    PRIMEM["Greenwich",0,
        ANGLEUNIT["degree",0.0174532925199433]],
    CS[ellipsoidal,2],
        AXIS["geodetic latitude (Lat)",north,
            ORDER[1],
            ANGLEUNIT["degree",0.0174532925199433]],
        AXIS["geodetic longitude (Lon)",east,
            ORDER[2],
            ANGLEUNIT["degree",0.0174532925199433]],
    ID["EPSG",4326]]
Data axis to CRS axis mapping: 2,1
Origin = (38.499974999999999,-3.499875000000000)
Pixel Size = (0.000269933742331,-0.000269933742331)
Metadata:
  AREA_OR_POINT=Area
Image Structure Metadata:
  INTERLEAVE=PIXEL
Corner Coordinates:
Upper Left  (  38.4999750,  -3.4998750) ( 38d29'59.91"E,  3d29'59.55"S)
Lower Left  (  38.4999750,  -4.5998550) ( 38d29'59.91"E,  4d35'59.48"S)
Upper 

----  
Check to see what files exist in `geotiffs`

In [28]:
!ls -lah geotiffs/*.tif

-rw-rw-r-- 1 ksaadmin ksaadmin 1.1G Jun  2 08:50 geotiffs/landsat7_2000_12_07_07_21_56.tif
-rw-rw-r-- 1 ksaadmin ksaadmin 1.1G Jun  2 08:50 geotiffs/landsat7_2002_11_11_07_19_30.tif
-rw-rw-r-- 1 ksaadmin ksaadmin 1.1G Jun  2 08:50 geotiffs/landsat7_2004_09_29_07_20_12.tif
-rw-rw-r-- 1 ksaadmin ksaadmin 1.1G Jun  2 08:50 geotiffs/landsat7_2006_02_23_07_21_42.tif


----  
Zip all geotiffs  

In [ ]:
!tar -cvzf geotiffs/landsat_7.tar.gz geotiffs/*.tif

geotiffs/landsat7_2000_12_07_07_21_56.tif
geotiffs/landsat7_2002_11_11_07_19_30.tif
geotiffs/landsat7_2004_09_29_07_20_12.tif
geotiffs/landsat7_2006_02_23_07_21_42.tif


----  
List files again to see the size of the gif created

In [ ]:
!ls -lah geotiffs/

#Inspect the folders as well